In [179]:
import warnings

warnings.filterwarnings('ignore')

In [180]:
DATA_FOLDER = '../data/'
VARIABLE_FOLDER = '../variables/'
FIGURE_FOLDER = '../reports/'

In [181]:
import pandas as pd

# df = pd.read_csv(DATA_FOLDER + '/processed/posts_and_comments.csv')
df = pd.read_csv(DATA_FOLDER + 'processed/assigned_topics.csv')

In [182]:
# timestamps = df.date.to_list()
# classes = df['query'].to_list()
texts_list = df.content.to_list()

In [184]:
# post_word_freq_dicts[:3]

In [185]:
from transformers import pipeline

sentiment_analysis = pipeline("text-classification", model="j-hartmann/sentiment-roberta-large-english-3-classes",
                              return_all_scores=True)

Some weights of the model checkpoint at j-hartmann/sentiment-roberta-large-english-3-classes were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [186]:
import numpy as np
import re

def find_string_sentiment(text, query):
    len_text = len(text)
    words = text.split()  # Split the text into words
    print(len(words))

    sents_tokens = []  # List to store sentiment scores for individual tokens

    for word in words:
        sentiment_i = sentiment_analysis(word)  # Analyze sentiment using the pipeline
        sentiment_scores = sentiment_i[0]  # Get the sentiment scores for the token
        sents_tokens.append(sentiment_scores)

    # print(sents_tokens)

    # Calculate sentiment encoding for tokens based on the highest score
    # sentiment_encoding_tokens = []
    # for sentiment_scores in sents_tokens:
    #     max_score_label = max(sentiment_scores, key=lambda x: x['score'])['label']
    #     if max_score_label == 'negative':
    #         sentiment_encoding_tokens.append(-1)
    #     elif max_score_label == 'neutral':
    #         sentiment_encoding_tokens.append(0)
    #     elif max_score_label == 'positive':
    #         sentiment_encoding_tokens.append(1)
    #
    # print(len(sentiment_encoding_tokens))
    # # Calculate average sentiment scores for tokens
    # avg_sent_tokens = np.mean(sentiment_encoding_tokens)
    # print(avg_sent_tokens)
    #
    # # Analyze sentiment for the entire phrase using the pipeline
    # sentiment_phrase = sentiment_analysis(text)
    # sentiment_phrase_scores = sentiment_phrase[0]  # Get the sentiment scores for the phrase
    # max_score_label = max(sentiment_phrase_scores, key=lambda x: x['score'])['label']
    # if max_score_label == 'negative':
    #     avg_sent_phrase = -1
    # elif max_score_label == 'neutral':
    #     avg_sent_phrase = 0
    # elif max_score_label == 'positive':
    #     avg_sent_phrase = 1
    #
    # print(max_score_label)

    return words, sents_tokens

# def find_string_sentiment(text, query):
#     len_text = len(text)
#
#     # index = text.lower().find(query)
#     indices = [match.start() for match in re.finditer(query, text.lower())]
#
#     sents = np.empty((len(indices), 3))
#
#     for i, index in enumerate(indices):
#
#         if (index - 256) < 0:
#             start_idx = 0
#         else:
#             start_idx = (index - 256)
#
#         if (index + 256) > len_text:
#             end_idx = len_text
#         else:
#             end_idx = (index + 256)
#
#         sentiment_i = sentiment_analysis(text[start_idx:end_idx])
#         # print(sentiment_i[0][0])
#         sents[i, :] = sentiment_i[0][0]['score'], sentiment_i[0][1]['score'], sentiment_i[0][2]['score']
#
#     return np.mean(sents, axis=0)


In [188]:
import pandas as pd

# from tqdm import tqdm

sentiments = pd.DataFrame()

lexicon = {}  # Initialize the lexicon dictionary

for query in df['query'].unique():
    query_data = df[df['query'] == query]  # Subset DataFrame for the current query
    largest_post = query_data.loc[query_data['content'].apply(len).idxmax()]  # Get the largest post
    try:
        text = largest_post['content']
        words, sent_tokens = find_string_sentiment(text, query)  # Apply sentiment analysis to the largest post
        sentiments[query] = {'words': words, 'sent_tokens': sent_tokens}
        # print(f"Processed query: {query}")
        for word, sentiment_scores in zip(words, sent_tokens):
            # print(sentiment_scores)
            max_score_label = max(sentiment_scores, key=lambda x: x['score'])['label']
            if word not in lexicon:
                if max_score_label == 'neutral':
                    lexicon[word] = 0
                elif max_score_label == 'negative':
                    lexicon[word] = sentiment_scores[0]['score'] * -1
                else:
                    lexicon[word] = sentiment_scores[2]['score']  # Positive score remains unchanged
                # print(lexicon[word])
        # print(i, end="\r")
    except Exception as e:
        print(e)
        continue


# for i, text in (enumerate(texts_list)):
#     try:
#         sentiments.loc[i, :] = find_string_sentiment(text, df.iloc[i]['query'])
#         print(i, end="\r")
#     except:
#         continue


4957
Processed query: semaglutide
1400
Processed query: ozempic
2290
Processed query: wegovy
2458
Processed query: saxenda
1505
Processed query: tirzepatide
2138
Processed query: mounjaro
4957
Processed query: GLP-1
4957
Processed query: liraglutide
2290
Processed query: victoza
4957
Processed query: GLP1
4957
Processed query: exenatide
4957
Processed query: lixisenatide
1025
Processed query: bydureon
1635
Processed query: trulicity
254
Processed query: orforglipron
948
Processed query: byetta
2290
Processed query: rybelsus
585
Processed query: dulaglutide
166
Processed query: GLP1RA
323
Processed query: bydureon bcise
181
Processed query: adlyxin
294
Processed query: retatrutide
232
Processed query: ly3437943
1058
Processed query: GLP-1RA


In [ ]:
lexicon

In [ ]:
import matplotlib.pyplot as plt

# Filter out entries with a score of 0 and sort by magnitude of score
filtered_lexicon = {word: score for word, score in lexicon.items() if score != 0}
sorted_lexicon = dict(sorted(filtered_lexicon.items(), key=lambda item: abs(item[1]), reverse=True))

# Extract keys and values from the sorted lexicon dictionary
words = list(sorted_lexicon.keys())
scores = list(sorted_lexicon.values())

# Create a bar chart centered around 0
plt.barh(words, scores, color=['red' if score < 0 else 'green' for score in scores])

# Add a vertical line at 0
plt.axvline(x=0, color='black', linewidth=0.8)

# Customize plot labels and title
plt.xlabel('Sentiment Score')
plt.ylabel('Words')
plt.title('Sentiment Scores of Words (sorted by magnitude)')
# plt.grid(axis='x', linestyle='--', alpha=0.7)

# Show the plot
plt.savefig(FIGURE_FOLDER + 'figures/Sentiment Analysis.png', bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
sentiment_map = {0: 'negative', 1: 'neutral', 2: 'positive'}
sentiments['net'] = np.argmax(np.array(sentiments.loc[:, ['negative', 'neutral', 'positive']]), axis=1)
# sentiments['net'] = sentiments['net'].apply(lambda x : sentiment_map[x])

In [ ]:
sentiments.index = df.index
# sentiments = df.join(sentiments)
# sentiments = pd.read_csv('sentiments.csv')
# sentiments.to_csv(DATA_FOLDER + 'processed/sentiments.csv')

In [ ]:
sentiments['net_t'] = sentiments['net'] - 1

In [ ]:
# df_topics = pd.read_csv(DATA_FOLDER + '/processed/assigned_topics.csv')

In [ ]:
sentiments['topic'] = df['Topic']

In [ ]:
sentiments = df.merge(sentiments, left_index=True, right_index=True)

In [ ]:
sentiments.to_csv(DATA_FOLDER + 'processed/sentiments.csv')